In [2]:
# Erstellt Einträge in der Datenbank-Tabelle broccoliBlacklist. Diese Datensätze werden später ignoriert
import pyodbc
import pandas as pd
import glob

# CSV einlesen und Datensätze filtern, die in DB geschrieben werden sollen
date = '2019-05-16'
if date == '':
    csv_files = glob.glob("pixelCount_notableIDs_*.csv")
else:
    csv_files = glob.glob("pixelCount_notableIDs_" + date + ".csv")

blacklist = pd.concat([pd.read_csv(f, delimiter=';') for f in csv_files], ignore_index = True, sort=False)
blacklist = blacklist[blacklist.toBlacklist == 1]
blacklist

,id,pixelCount,toBlacklist
22,633,64,1
25,762,129,1
26,773,114,1
28,797,110,1
29,835,185,1
30,883,108,1
32,918,50,1
36,1072,87,1
39,1136,102,1
40,1216,106,1


In [3]:
# Azure-DB Verbindungsaufbau und INSERT
server = 'deepbroccoliserver.database.windows.net'
database = 'DeepBroccoliDatabase'
username = 'ntb'
password = 'brokkoli_2019'
driver= '{SQL Server}'
conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

for index, row in blacklist.iterrows():    
    cursor.execute('''
                    IF NOT EXISTS (SELECT * FROM dbo.broccoliBlacklist WHERE broccoli_id = ''' + str(row['id']) + ''')
                    BEGIN
                        INSERT INTO dbo.broccoliBlacklist (broccoli_id, reason)
                        VALUES (''' + str(row['id']) + ', \'EDA\'' + ''')
                    END
                    ''')

conn.commit()

SQL_Query = pd.read_sql_query(''' select * from dbo.broccoliBlacklist''', conn)
sqlBlacklist = pd.DataFrame(SQL_Query)
sqlBlacklist

,broccoli_id,reason
0,282,EDA
1,633,EDA
2,762,EDA
3,773,EDA
4,797,EDA
5,835,EDA
6,883,EDA
7,918,EDA
8,1072,EDA
9,1136,EDA
